_unchecked_

# CNTK 202: 语言理解与复发的网络

本教程演示如何实现一个用于处理文本的经常性网络, 用于为[航空旅行信息服务](https://catalog.ldc.upenn.edu/LDC95S26)(自动分类) 任务添加插槽标记 (将单个单词标记为各自的类, 在培训数据中作为标签提供这些类集)。

本教程有2部分:-1 部分: 我们将每个单词按顺序标记到相应的标签-2 部分: 我们将把一个序列分类为其相应的意图。

我们将从一个直向前 (线性) 嵌入的词, 然后是一个经常性的 LSTM, 以标签的每个字序列到相应的类。我们将展示如何将序列中的每个单词标记分类到相应的类。然后将其扩展到包含相邻单词并运行双向。

我们将采用序列的最后一个状态, 并训练一个模型, 将整个序列分类为相应的类标签 (在本例中是与序列关联的意图)。

您将练习的技术是:*通过构成层块的模型描述, 一种在不需要编写公式的情况下编写网络/模型的便捷方法,*创建您自己的层块*具有不同序列长度的变量同一个网络*培训网络

我们假设您熟悉深层学习的基础知识, 以及这些具体的概念:*经常性网络 (*文本嵌入 ([维基百科页](https://en.wikipedia.org/wiki/Word_embedding))

## 先决条件

我们假定您已经[安装了 CNTK](https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-CNTK-on-your-machine)。
本教程要求 CNTK V2。我们强烈建议在具有 CUDA 兼容的 GPU 的机器上运行本教程。没有 gpu 的深入学习是不好玩的。

_unchecked_

## 数据下载

在本教程中, 我们将使用一个 (轻度预处理) 版本的数据分析。您可以通过运行下面的单元格或执行手动指令来自动下载数据。

**备用手动指令**
下载[培训](https://github.com/Microsoft/CNTK/blob/release/2.4/Tutorials/SLUHandsOn/atis.train.ctf)和[测试](https://github.com/Microsoft/CNTK/blob/release/2.4/Tutorials/SLUHandsOn/atis.test.ctf)文件, 并将它们放在与此笔记本相同的文件夹中。如果你想看到模型是如何预测新的句子, 你还需要[查询](https://github.com/Microsoft/CNTK/blob/release/2.4/Examples/LanguageUnderstanding/ATIS/BrainScript/query.wl)和[插槽](https://github.com/Microsoft/CNTK/blob/release/2.4/Examples/LanguageUnderstanding/ATIS/BrainScript/slots.wl)的词汇文件

In [1]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import requests
import os

def download(url, filename):
    """ utility function to download a file """
    response = requests.get(url, stream=True)
    with open(filename, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

locations = ['Tutorials/SLUHandsOn', 'Examples/LanguageUnderstanding/ATIS/BrainScript']

data = {
  'train': { 'file': 'atis.train.ctf', 'location': 0 },
  'test': { 'file': 'atis.test.ctf', 'location': 0 },
  'query': { 'file': 'query.wl', 'location': 1 },
  'slots': { 'file': 'slots.wl', 'location': 1 },
  'intent': { 'file': 'intent.wl', 'location': 1 }  
}

for item in data.values():
    location = locations[item['location']]
    path = os.path.join('..', location, item['file'])
    if os.path.exists(path):
        print("Reusing locally cached:", item['file'])
        # Update path
        item['file'] = path
    elif os.path.exists(item['file']):
        print("Reusing locally cached:", item['file'])
    else:
        print("Starting download:", item['file'])
        url = "https://github.com/Microsoft/CNTK/blob/release/2.4/%s/%s?raw=true"%(location, item['file'])
        download(url, item['file'])
        print("Download completed")


Reusing locally cached: query.wl
Reusing locally cached: intent.wl
Reusing locally cached: atis.test.ctf
Reusing locally cached: atis.train.ctf
Reusing locally cached: slots.wl


_unchecked_

**导入库**: CNTK、数学和 numpy

CNTK 的 Python 模块包含多个子, 如 `io` 、 `learner` 和 `layers` 。在某些情况下, 我们也使用 NumPy, 因为 CNTK 返回的结果类似于 NumPy 数组。

In [2]:
import math
import numpy as np

import cntk as C
import cntk.tests.test_utils
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components

_unchecked_

## 任务概述: 插槽标记

我们要在本教程中处理的任务是插槽标记。
我们使用 "it[语料库](https://catalog.ldc.upenn.edu/LDC95S26)"。
它包含了航空旅行信息服务领域的人机查询, 我们的任务是对查询的每个单词进行注释 (标记), 无论它属于特定的信息项目 (插槽), 还是哪一个。

工作文件夹中的数据已被转换为 "CNTK 文本格式"。
让我们看看测试集文件 `atis.test.ctf` 中的一个示例:

    `19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O
    `

此文件有7列:

- 序列 id (19)。有11条目与这个序列 id。这意味着序列19由11标记组成;

- 列 `S0` , 其中包含数字字索引; 输入数据在一个热向量中编码。 在词汇表中有943单词, 所以每个单词都是一个943元向量, 所有的0都有一个1的向量索引来表示这个词。 例如词 "从" 代表以1在索引444和零在传染媒介的其他到处。词 "星期一" 代表以 a 1 在索引601和零在所有其他在传染媒介。

- 由 `#` 表示的注释列, 允许人类读者知道数字单词索引代表什么;
系统将忽略注释列。`BOS`and `EOS` 是特殊的词, 分别表示句子的开头和结尾;

- 列 `S1` 是意向标签, 我们将在本教程的第二部分中使用它;

- 另一个显示数字意图索引的可读标签的注释列;

- 列 `S2` 是插槽标签, 表示为数字索引;

- 另一个显示数字标签索引的可读标签的注释列。

神经网络的任务是查看查询 (列 `S0` ) 并预测插槽标签 (列 `S2` )。
正如您所看到的, 输入中的每个单词都为第一个单词指定了一个空标签 `O` 或一个以 `B-` 开头的插槽标签, 以及 `I-` 对于属于同一插槽的任何其他连续单词。

### 模型创建

我们将使用的模型是一个重复的模型, 包括一个嵌入层, 一个复发的 LSTM 细胞, 和一个稠密的层来计算后的概率:

    `slot label   "O"        "O"        "O"        "O"  "B-fromloc.city_name"
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Dense |  | Dense |  | Dense |  | Dense |  | Dense |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"
    `

或者, 作为一个 CNTK 的网络描述。请快速查看并与上面的描述相匹配: (这些函数的说明可以在[层引用](http://cntk.ai/pythondocs/layerref.html)中找到)

In [3]:
# number of words in vocab, slot labels, and intent labels
vocab_size = 943 ; num_labels = 129 ; num_intents = 26    

# model dimensions
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 150
hidden_dim = 300

# Create the containers for input feature (x) and the label (y)
x = C.sequence.input_variable(vocab_size)
y = C.sequence.input_variable(num_labels)

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels, name='classify')
        ])

_unchecked_

现在, 我们准备创建一个模型并对其进行检查。

模型属性可从 Python 中完全访问。名为 `embed` 的第一层是嵌入层。在这里, 我们使用 CNTK 默认, 这是线性嵌入。它是一个简单的矩阵与尺寸 (输入字编码 x 输出投影维度)。您可以像 Python 对象的任何其他属性一样访问其参数 `E` (存储嵌入)。它的形状包含一个 `-1` , 它指示此参数 (带有输入维度) 尚未完全指定, 而输出维度设置为 `emb_dim` (本教程中为 150)。

此外, 我们还检查 `Dense` 层名 `classify` 中的偏置向量的值。`Dense`层是多层感知器的基本组成单元 (如 CNTK 103C 教程中介绍的)。`Dense`层同时具有 `weight` 和 `bias` 参数, 每个 `Dense` 层都有一个。默认情况下, 缺省值初始化为 0 (但如果需要, 还有一种方法可以更改)。在创建模型时, 应该命名层组件, 然后访问这些参数, 如下所示。

**建议的任务**: `weight` 从名为的层中, 矩阵的预期维度应该是多少 `classify` ？尝试打印 `classify` 图层的权重矩阵？它与你的预期大小相符吗？

In [4]:
# peek
z = create_model()
print(z.embed.E.shape)
print(z.classify.b.value)

(-1, 150)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.]


_unchecked_

在我们的情况下, 我们有输入作为一个热编码向量的长度943和输出维度 `emb_dim` 设置为150。在下面的代码中, 我们将输入变量 `x` 传递给模型 `z` 。这将使用已知形状的输入数据绑定模型。在这种情况下, 输入的形状将是输入词汇的大小。通过此修改, 嵌入层返回的参数完全指定 (943、150)。**注意**: 您可以使用[Word2Vec](https://en.wikipedia.org/wiki/Word2vec)或[手套](https://en.wikipedia.org/wiki/GloVe_%28machine_learning%29)初始化带有预计算向量的嵌入矩阵。

In [5]:
# Pass an input and check the dimension
z = create_model()
print(z(x).embed.E.shape)

(943, 150)


_unchecked_

为了在 CNTK 中训练和测试模型, 我们需要创建一个模型, 并指定如何读取数据和执行培训和测试。

为了训练我们需要指定:

- 如何读取数据

- 模型函数、输入和输出

- 学生的超参数, 如学习率

## 数据读取

我们已经看过这些数据了。
但如何生成此格式？
对于阅读文本, 本教程使用 `CNTKTextFormatReader` 。它希望输入数据以特定的格式, 如[此处](https://docs.microsoft.com/en-us/cognitive-toolkit/Brainscript-CNTKTextFormat-Reader)所述。

在本教程中, 我们通过两个步骤创建了语料库: * 将原始数据转换为包含空格分隔文本的制表符分隔列的纯文本文件。例如:

`BOS show flights from burbank to st. louis on monday EOS (TAB) flight (TAB) O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.day_name O`

这意味着与 `paste` 命令的输出兼容. * 使用以下命令将其转换为 CNTK 文本格式 (周大福):

`python [CNTK root]/Scripts/txt2ctf.py --map query.wl intent.wl slots.wl --annotated True --input atis.test.txt --output atis.test.ctf`其中, 三 `.wl` 文件将词汇表作为纯文本文件提供, 每行一个单词。

在这些周大福文件中, 我们的列被标记为 `S0` 、 `S1` 和 `S2` 。
它们通过读取器定义中的相应行连接到实际的网络输入:

In [6]:
def create_reader(path, is_training):
    return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
         query         = C.io.StreamDef(field='S0', shape=vocab_size,  is_sparse=True),
         intent        = C.io.StreamDef(field='S1', shape=num_intents, is_sparse=True),  
         slot_labels   = C.io.StreamDef(field='S2', shape=num_labels,  is_sparse=True)
     )), randomize=is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)

In [7]:
# peek
reader = create_reader(data['train']['file'], is_training=True)
reader.streams.keys()

dict_keys(['query', 'slot_labels', 'intent'])

_unchecked_

## 培训

我们还必须定义培训标准 (损失函数), 以及跟踪的误差度量。在大多数教程中, 我们知道输入维度和相应的标签。我们直接创建的损失和错误的功能。在本教程中, 我们将这样做。然而, 我们采取简短的绕道, 并了解占位符。这个概念对于任务3是有用的。

**学习注意事项**: `placeholder` 请记住, 我们编写的代码实际上并没有执行任何繁重的计算, 它只是在训练/测试过程中指定我们要计算的数据的函数。同样, 当在编程语言中编写常规函数时, 为参数命名是方便的, 因此, 使用引用参数的占位符 (或需要重用的本地计算) 很方便。最终, 其他一些代码将用其他已知的数量替换这些占位符, 就像在编程语言中一样, 函数将用与其参数绑定的具体值来调用。

具体地说, 您在上面创建的输入变量 `x = C.sequence.input_variable(vocab_size)` 保存预先定义的数据 `vocab_size` 。如果此类实例化具有挑战性或不可能, 则使用 `placeholder` 是一个合乎逻辑的选择。具有 `placeholder` 仅允许您在可能有数据的稍后时间推迟参数的规范。

下面的示例说明了 `placeholder` 的用法。

In [8]:
def create_criterion_function(model):
    labels = C.placeholder(name='labels')
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    return C.combine ([ce, errs]) # (features, labels) -> (loss, metric)

criterion = create_criterion_function(create_model())
criterion.replace_placeholders({criterion.placeholders[0]: C.sequence.input_variable(num_labels)})

Composite(Combine): Input('Input2300', [#, *], [129]), Placeholder('labels', [???], [???]) -> Output('Block2270_Output_0', [#, *], [1]), Output('Block2290_Output_0', [#, *], [])

_unchecked_

当上面的单元格在网络创建中定义了输入参数时效果很好, 这会损害可读性。因此, 我们更喜欢创建如下所示的函数

In [9]:
def create_criterion_function_preferred(model, labels):
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    return ce, errs # (model, labels) -> (loss, error metric)

In [10]:
def train(reader, model_func, max_epochs=10, task='slot_tagging'):
    
    # Instantiate the model function; x is the input (feature) variable 
    model = model_func(x)
    
    # Instantiate the loss and error function
    loss, label_error = create_criterion_function_preferred(model, y)

    # training config
    epoch_size = 18000        # 18000 samples is half the dataset size 
    minibatch_size = 70
    
    # LR schedule over epochs 
    # In CNTK, an epoch is how often we get out of the minibatch loop to
    # do other stuff (e.g. checkpointing, adjust learning rate, etc.)
    lr_per_sample = [3e-4]*4+[1.5e-4]
    lr_per_minibatch = [lr * minibatch_size for lr in lr_per_sample]
    lr_schedule = C.learning_parameter_schedule(lr_per_minibatch, epoch_size=epoch_size)
    
    # Momentum schedule
    momentums = C.momentum_schedule(0.9048374180359595, minibatch_size=minibatch_size)
    
    # We use a the Adam optimizer which is known to work well on this dataset
    # Feel free to try other optimizers from 
    # https://www.cntk.ai/pythondocs/cntk.learner.html#module-cntk.learner
    learner = C.adam(parameters=model.parameters,
                     lr=lr_schedule,
                     momentum=momentums,
                     gradient_clipping_threshold_per_sample=15, 
                     gradient_clipping_with_truncation=True)

    # Setup the progress updater
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=max_epochs)
    
    # Uncomment below for more detailed logging
    #progress_printer = ProgressPrinter(freq=100, first=10, tag='Training', num_epochs=max_epochs) 

    # Instantiate the trainer
    trainer = C.Trainer(model, (loss, label_error), learner, progress_printer)

    # process minibatches and perform model training
    C.logging.log_number_of_parameters(model)
    
    # Assign the data fields to be read from the input
    if task == 'slot_tagging':
        data_map={x: reader.streams.query, y: reader.streams.slot_labels}
    else:
        data_map={x: reader.streams.query, y: reader.streams.intent} 

    t = 0
    for epoch in range(max_epochs):         # loop over epochs
        epoch_end = (epoch+1) * epoch_size
        while t < epoch_end:                # loop over minibatches on the epoch
            data = reader.next_minibatch(minibatch_size, input_map= data_map)  # fetch minibatch
            trainer.train_minibatch(data)               # update model with it
            t += data[y].num_samples                    # samples so far
        trainer.summarize_training_progress()

_unchecked_

**运行培训师**

你可以找到完整的食谱下面。

In [11]:
def do_train():
    global z
    z = create_model()
    reader = create_reader(data['train']['file'], is_training=True)
    train(reader, z)
do_train()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.740198 * 18010, metric = 28.02% * 18010 6.466s (2785.3 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.665177 * 18051, metric = 14.30% * 18051 5.238s (3446.2 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.526256 * 17941, metric = 11.34% * 17941 5.198s (3451.5 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.395405 * 18059, metric = 8.22% * 18059 5.329s (3388.8 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.293512 * 17957, metric = 6.20% * 17957 5.106s (3516.8 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.264932 * 18021, metric = 5.73% * 18021 5.335s (3377.9 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.217258 * 17980, metric = 4.69% * 17980 5.248s (3426.1 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.209614 * 18025, metric =

_unchecked_

这显示了学习如何在时代 (通过数据)。
例如, 在四个世纪以后, 损失, 是交叉熵标准, 减少了显着测量在这个世纪的 ~ 18000 样品, 和同样与错误率在那些同样18000训练样品。

纪元大小是在模型检查点之间处理的样本数--计算为*单词标记*, 而不是句子。

一旦训练完成 (在一个泰坦 x 或表面书上不到2分钟), 你将看到一个像这样的输出 (交叉熵) 和度量 (分类错误) 平均在最后的纪元。

在仅用于 CPU 的计算机上, 它的速度可能会减慢4倍或更慢。您可以尝试设置 `python
emb_dim    = 50 
hidden_dim = 100` 以减少在 CPU 上运行所需的时间, 但模型将不适合以及隐藏和嵌入维度较大时的状态。

_unchecked_

### 评估模型

与火车 () 功能一样, 我们还定义了一个函数, 通过计算测试数据的多个 minibatches 的误差来测量测试集的精度。为了对从文件读取的小样本进行评估, 可以设置反映样本大小的 minibatch 大小, 并在该数据实例上运行 test_minibatch。要了解如何评估单个序列, 我们将在后面的教程中提供一个实例。

In [12]:
def evaluate(reader, model_func, task='slot_tagging'):
    
    # Instantiate the model function; x is the input (feature) variable 
    model = model_func(x)
    
    # Create the loss and error functions
    loss, label_error = create_criterion_function_preferred(model, y)

    # process minibatches and perform evaluation
    progress_printer = C.logging.ProgressPrinter(tag='Evaluation', num_epochs=0)
    
    # Assign the data fields to be read from the input
    if task == 'slot_tagging':
        data_map={x: reader.streams.query, y: reader.streams.slot_labels}
    else:
        data_map={x: reader.streams.query, y: reader.streams.intent} 

    while True:
        minibatch_size = 500
        data = reader.next_minibatch(minibatch_size, input_map= data_map)  # fetch minibatch
        if not data:                                 # until we hit the end
            break

        evaluator = C.eval.Evaluator(loss, progress_printer)
        evaluator.test_minibatch(data)
     
    evaluator.summarize_test_progress()

_unchecked_

现在, 我们可以通过遍历测试集中的所有示例并使用 `C.eval.Evaluator` 方法来测量模型的准确性。

In [13]:
def do_test():
    reader = create_reader(data['test']['file'], is_training=False)
    evaluate(reader, z)
do_test()
z.classify.b.value

Finished Evaluation [1]: Minibatch[1-23]: metric = 0.34% * 10984;


array([ -1.86572317e-02,  -8.51036515e-03,   1.38878925e-02,
        -1.95176266e-02,  -2.78977025e-03,  -1.23168388e-02,
        -6.16775267e-03,  -1.48158008e-02,  -5.82036236e-03,
        -2.99133137e-02,  -1.39552690e-02,  -2.11144108e-02,
        -1.13499342e-02,  -1.25011550e-02,  -8.19404377e-04,
        -5.26463473e-03,  -2.67275460e-02,  -1.80706571e-04,
        -3.97865893e-03,  -2.99989916e-02,  -1.00385472e-02,
        -6.81575621e-03,  -2.65348833e-02,  -2.01367699e-02,
        -2.63106022e-02,   4.22888156e-03,   5.74267423e-03,
        -1.56373512e-02,   7.71288527e-04,  -2.11508083e-03,
        -9.64712165e-03,  -1.98591035e-02,  -1.32136559e-02,
         7.97899254e-03,  -1.76088810e-02,   9.19441786e-03,
         1.30802142e-02,  -3.85359419e-03,   1.86733739e-03,
        -5.96518070e-03,  -3.07163727e-02,  -3.04672867e-03,
        -3.46868881e-04,  -1.29565294e-03,  -4.47260169e-03,
        -1.29292896e-02,  -1.05356863e-02,  -9.16024856e-03,
         6.08767197e-03,

_unchecked_

下面的代码块阐释了如何计算单个序列。此外, 我们还展示了如何使用 NumPy 数组传递信息。

In [14]:
# load dictionaries
query_wl = [line.rstrip('\n') for line in open(data['query']['file'])]
slots_wl = [line.rstrip('\n') for line in open(data['slots']['file'])]
query_dict = {query_wl[i]:i for i in range(len(query_wl))}
slots_dict = {slots_wl[i]:i for i in range(len(slots_wl))}

# let's run a sequence through
seq = 'BOS flights from new york to seattle EOS'
w = [query_dict[w] for w in seq.split()] # convert to word indices
print(w)
onehot = np.zeros([len(w),len(query_dict)], np.float32)
for t in range(len(w)):
    onehot[t,w[t]] = 1

#x = C.sequence.input_variable(vocab_size)
pred = z(x).eval({x:[onehot]})[0]
print(pred.shape)
best = np.argmax(pred,axis=1)
print(best)
list(zip(seq.split(),[slots_wl[s] for s in best]))

[178, 429, 444, 619, 937, 851, 752, 179]
(8, 129)
[128 128 128  48 110 128  78 128]


[('BOS', 'O'),
 ('flights', 'O'),
 ('from', 'O'),
 ('new', 'B-fromloc.city_name'),
 ('york', 'I-fromloc.city_name'),
 ('to', 'O'),
 ('seattle', 'B-toloc.city_name'),
 ('EOS', 'O')]

_unchecked_

## 修改模型

在下面的内容中, 您将被赋予任务来练习修改 CNTK 配置。
本文件末尾给出了解决方案..。

**一个词关于**

在跳转到任务之前, 让我们再看看我们刚刚运行的模型。
该模型在我们所说的*函数-组合样式*中进行了描述。
`python
        Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])`您可能熟悉其他神经网络工具箱中的 "顺序" 符号。
如果不是, 则[ ](https://www.cntk.ai/pythondocs/layerref.html#sequential)是一个功能强大的操作, 简而言之, 它允许在神经网络中紧凑地表达一个非常常见的情况, 即通过通过层进程传播来处理输入. `Sequential()` 将函数列表作为其参数,
并返回一个*new*函数, 它按顺序调用这些函数, 每次将一个输出传递到下一个。
例如, 与 `python
    FGH = Sequential ([F,G,H])
    y = FGH (x)` 相同, `y = H(G(F(x)))` 这称为["函数组合"](https://en.wikipedia.org/wiki/Function_composition), 对于表达神经网络 (通常具有此形式) 尤其方便:

    `     +-------+   +-------+   +-------+
    x -->|   F   |-->|   G   |-->|   H   |--> y
         +-------+   +-------+   +-------+
    `

回到我们手头的模型, `Sequential` 表达式简单地说, 我们的模型具有以下形式:

    `     +-----------+   +----------------+   +------------+
    x -->| Embedding |-->| Recurrent LSTM |-->| DenseLayer |--> y
         +-----------+   +----------------+   +------------+
    `

_unchecked_

### 任务 1: 添加批处理规范化

我们现在要向模型中添加新层, 特别是批量规范化。

批量规范化是加速收敛的常用技术。
它通常用于图像处理设置。但它也能为经常性的模型工作吗？

>注意: 目前仅在 GPU 上支持批量规范化培训。

因此, 您的任务将是在重复的 LSTM 层之前和之后插入批处理规范化层。
如果您已完成了[图像处理上的动手实验室](https://github.com/Microsoft/CNTK/blob/release/2.4/Tutorials/CNTK_201B_CIFAR-10_ImageHandsOn.ipynb), 您可能还记得[批处理正常化层](https://www.cntk.ai/pythondocs/layerref.html#batchnormalization-layernormalization-stabilizer)具有此窗体: `BatchNormalization()` 所以请继续并修改配置, 看看会发生什么。

如果一切顺利, 您将注意到与以前的配置相比, 改进的收敛速度 ( `loss` and `metric` )。

In [15]:
# Your task: Add batch normalization
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])

# Enable these when done:
#do_train()
#do_test()

_unchecked_

### 任务 2: 添加前瞻

我们的经常性模式遭受结构性赤字: 由于定期运行从左向右, 因此对插槽标签的决策没有关于即将出现的单词的信息。这个模型有点不平衡。
您的任务将是修改模型, 这样, 循环的输入不仅包括当前单词, 而且还包含下一个 (前瞻性)。

您的解决方案应在功能组合样式中。
因此, 您将需要编写一个 Python 函数来执行以下操作:

- 不接受输入参数

- 创建占位符 (序列) 变量

- 使用 `sequence.future_value()` 操作, 计算此序列中的 "下一个值", 并

- 将当前和下一个值串联为两次嵌入维度的向量, 使用`splice()`

然后将该函数插入到嵌入层之后的 `Sequential()` 列表中。

In [16]:
# Your task: Add lookahead
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])
    
# Enable these when done:
#do_train()
#do_test()

_unchecked_

### 任务 3: 双向递归模型

哈哈, 对未来的语言有帮助的知识。因此, 不是一个字的前瞻, 为什么不向前看, 直到所有的方式到年底的句子, 通过向后复发？
让我们创建一个双向模型!

您的任务是实现一个新层, 它对数据执行向前和向后递归, 并连接输出向量。

但是, 请注意, 这与前面的任务不同, 因为双向层包含学习模型参数。
在函数-组合样式中, 实现带有模型参数的层的模式是编写创建*函数对象*的*工厂函数*。

函数对象 (也称为 "[None](https://en.wikipedia.org/wiki/Function_object)函数") 是一个既是函数又是对象的对象。
这意味着它所包含的数据还可以被调用, 就好像它是一个函数一样。

例如, `Dense(outDim)` 是一个工厂函数, 它返回一个包含权重矩阵的函数对象 `W` 、一个偏置 `b` 和另一个要计算的函数 `input @ W + b.` (这是使用[Python 3.5 表示法进行矩阵乘法](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-465)。
在 Numpy 语法中, 它是 `input.dot(W) + b` )。
例如, `Dense(1024)` 将创建这个函数对象, 它可以像其他任何函数一样立即使用: `Dense(1024)(x)` 。

让我们看一个进一步清晰的例子: 让我们实现一个将线性层与后续的批规范化相结合的新层。
为了允许函数组合, 需要将层实现为工厂函数, 这可能类似于以下内容:

`python
def DenseLayerWithBN(dim):
    F = Dense(dim)
    G = BatchNormalization()
    x = placeholder()
    apply_x = G(F(x))
    return apply_x`

调用此工厂函数将创建 `F` 、 `G` 、 `x` 和 `apply_x` 。在此示例中, `F` 和 `G` 是函数对象本身, `apply_x` 是要应用于数据的函数。
因此, 例如, 调用 `DenseLayerWithBN(1024)` 将创建一个包含线性层函数对象的对象 (称为 `F` )、一个批处理规范化函数对象 `G` 和 `apply_x` , 它是使用 `F` 和 `G` 实现此层的实际操作的函数。然后它将返回 `apply_x` 。到外部, `apply_x` 外观和行为类似于函数。但是, 在引擎盖下, `apply_x` 保留对其特定实例的访问 `F` 和 `G` 。

现在回到我们手头的任务。您现在需要创建一个工厂功能, 非常像上面的例子。
您应创建一个工厂函数, 创建两个递归层实例 (一个向前, 一个向后), 然后定义一个将两个层实例应用到同一 `x` 的 `apply_x` 函数, 并将这两个结果串联在一起。

好吧, 试试看!要知道如何在 CNTK 中实现向后递归, 请从如何进行转发递归中得到提示。
还请执行以下操作:*删除在上一任务中添加的单字词前瞻性, 我们希望替换该项目; 并*确保每个 LSTM 都使用 `hidden_dim//2` 输出来保持模型参数的总数量受到限制。

In [17]:
# Your task: Add bidirectional recurrence
def create_model():
    with C.layers.default_options(initial_state=0.1):  
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])

# Enable these when done:
#do_train()
#do_test()

_unchecked_

双向模型的参数比预测先行的少40%。但是, 如果你回去仔细看, 你会发现前瞻一训练约30% 更快。
这是因为前瞻模型的水平依赖性 (而不是两次重复) 和更大的矩阵产品都具有较低的并行性, 因此可以实现更高的平行度。

_unchecked_

**解决方案 1: 添加批处理规范化**

In [18]:
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            #C.layers.BatchNormalization(), #Remove this comment if running on GPU
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            #C.layers.BatchNormalization(), #Remove this comment if running on GPU
            C.layers.Dense(num_labels)
        ])

do_train()
do_test()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.740198 * 18010, metric = 28.02% * 18010 4.223s (4264.7 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.665177 * 18051, metric = 14.30% * 18051 3.917s (4608.4 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.526256 * 17941, metric = 11.34% * 17941 3.898s (4602.6 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.395405 * 18059, metric = 8.22% * 18059 4.061s (4446.9 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.293512 * 17957, metric = 6.20% * 17957 3.996s (4493.7 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.264932 * 18021, metric = 5.73% * 18021 3.931s (4584.3 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.217258 * 17980, metric = 4.69% * 17980 3.941s (4562.3 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.209614 * 18025, metric =

_unchecked_

**解决方案 2: 添加前瞻**

In [19]:
def OneWordLookahead():
    x = C.placeholder()
    apply_x = C.splice(x, C.sequence.future_value(x))
    return apply_x

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            OneWordLookahead(),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)        
        ])

do_train()
do_test()

Training 901479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.618925 * 18010, metric = 26.40% * 18010 4.567s (3943.5 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.572762 * 18051, metric = 12.46% * 18051 5.560s (3246.6 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.420728 * 17941, metric = 8.57% * 17941 5.254s (3414.7 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.297996 * 18059, metric = 6.28% * 18059 5.697s (3169.9 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.224015 * 17957, metric = 4.81% * 17957 5.612s (3199.8 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.207126 * 18021, metric = 4.61% * 18021 5.487s (3284.3 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.170268 * 17980, metric = 3.69% * 17980 5.538s (3246.7 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.164910 * 18025, metric = 

_unchecked_

**解决方案 3: 双向递归模型**

In [20]:
def BiRecurrence(fwd, bwd):
    F = C.layers.Recurrence(fwd)
    G = C.layers.Recurrence(bwd, go_backwards=True)
    x = C.placeholder()
    apply_x = C.splice(F(x), G(x))
    return apply_x 

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            BiRecurrence(C.layers.LSTM(hidden_dim//2), 
                                  C.layers.LSTM(hidden_dim//2)),
            C.layers.Dense(num_labels)
        ])

do_train()
do_test()

Training 541479 parameters in 9 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.886776 * 18010, metric = 30.06% * 18010 7.619s (2363.8 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.683211 * 18051, metric = 14.83% * 18051 7.325s (2464.3 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.521379 * 17941, metric = 11.42% * 17941 7.265s (2469.5 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.394698 * 18059, metric = 8.11% * 18059 7.567s (2386.5 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.288926 * 17957, metric = 6.06% * 17957 7.386s (2431.2 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.267000 * 18021, metric = 5.73% * 18021 7.401s (2434.9 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.215379 * 17980, metric = 4.69% * 17980 7.269s (2473.5 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.206970 * 18025, metric =

_unchecked_

## 任务概述: 序列分类

我们将为这个任务重用相同的数据。我们再次访问一个示例:

    `19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O
    `

神经网络的任务是查看查询 (列 `S0` ) 并预测序列 (列 `S1` ) 的意图。这次我们将忽略插槽标签 (列 `S2` )。

### 模型创建

我们将使用的模型是一个经常性的模型, 包括嵌入层, 一个复发的 LSTM 细胞, 和一个稠密的层来计算后的概率。虽然非常类似的插槽标签模型在这种情况下, 我们只看在嵌入从最后一层:

    `intent                                                "flight"
                                                              ^          
                                                              |          
                                                          +-------+  
                                                          | Dense |   ...
                                                          +-------+  
                                                              ^         
                                                              |          
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"
    `

或者, 作为一个 CNTK 的网络描述。请快速查看并与上面的描述相匹配: (这些函数的说明可以在[层引用](http://cntk.ai/pythondocs/layerref.html)中找到)

#### 注意事项:

- 此模型与上一型号之间的第一个区别是关于标签 `y` 的规范。因为每个序列只有一个标签, 所以我们使用 `C.input_variable` 。

- 第二个区别是使用[稳定](http://ieeexplore.ieee.org/document/7472719/)器。我们稳定了嵌入式输出。稳定器为学习增加了一个额外的标量参数, 可以帮助我们的网络在训练中更快地收敛。

- 第三个区别是使用称为 `Fold` 的层函数。如上面的模型所示, 我们希望模型有 LSTM 复发, 除了最后一个, 我们建立了一个 LSTM 复发。最后的复发将是一个折叠操作, 我们从最后一个 LSTM 块中选取隐藏状态, 并将其用于整个序列的分类。

In [21]:
# number of words in vocab, slot labels, and intent labels
vocab_size = 943 ; num_intents = 26    

# model dimensions
emb_dim    = 150
hidden_dim = 300

# Create the containers for input feature (x) and the label (y)
x = C.sequence.input_variable(vocab_size)
y = C.input_variable(num_intents)

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Stabilizer(),
            C.layers.Fold(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_intents, name='classify')
        ])

_unchecked_

我们使用新模型创建 `criterion` 函数, 并相应地更新占位符。

In [22]:
criterion = create_criterion_function(create_model())
criterion.replace_placeholders({criterion.placeholders[0]: C.sequence.input_variable(num_intents)})

Composite(Combine): Input('Input14623', [#, *], [26]), Placeholder('labels', [???], [???]) -> Output('Block14593_Output_0', [#], [1]), Output('Block14613_Output_0', [#], [])

_unchecked_

除了在这种情况下我们提供 `intent` 标记作为标签的事实之外, 还可以使用相同的列车代码。

In [23]:
def do_train():
    global z
    z = create_model()
    reader = create_reader(data['train']['file'], is_training=True)
    train(reader, z, 5, 'intent')
do_train()

Training 690477 parameters in 7 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 5]: [Training] loss = 0.585308 * 18004, metric = 14.42% * 18004 79.421s (226.7 samples/s);
Finished Epoch[2 of 5]: [Training] loss = 0.153843 * 17998, metric = 3.92% * 17998 74.394s (241.9 samples/s);
Finished Epoch[3 of 5]: [Training] loss = 0.081964 * 18000, metric = 2.17% * 18000 59.060s (304.8 samples/s);
Finished Epoch[4 of 5]: [Training] loss = 0.069163 * 18000, metric = 1.92% * 18000 58.621s (307.1 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 5]: [Training] loss = 0.018180 * 17998, metric = 0.47% * 17998 58.345s (308.5 samples/s);


_unchecked_

现在, 我们可以通过遍历测试集中的所有示例并使用 C. eval 方法来测量模型的准确性。

In [24]:
def do_test():
    reader = create_reader(data['test']['file'], is_training=False)
    evaluate(reader, z, 'intent')
do_test()
z.classify.b.value

Finished Evaluation [1]: Minibatch[1-23]: metric = 0.00% * 893;


array([ 0.51183742, -0.29180217, -0.41856512, -0.28958377, -0.72160971,
       -0.24883319,  0.02916328, -0.0852626 ,  0.19014142, -0.41936243,
       -0.23639332, -0.29029393, -0.74258387, -0.12296562,  0.34665295,
       -0.46388549, -0.73981428, -0.2296015 , -0.78151304, -0.24418215,
       -0.52702737,  0.10101102, -0.27433836, -0.24181543, -0.39551306,
        0.30569023], dtype=float32)

_unchecked_

下面的代码块阐释了如何计算单个序列。此外, 我们还展示了如何使用 NumPy 数组传递信息。

In [25]:
# load dictionaries
query_wl = [line.rstrip('\n') for line in open(data['query']['file'])]
intent_wl = [line.rstrip('\n') for line in open(data['intent']['file'])]
query_dict = {query_wl[i]:i for i in range(len(query_wl))}
intent_dict = {intent_wl[i]:i for i in range(len(intent_wl))}

# let's run a sequence through
seq = 'BOS flights from new york to seattle EOS'
w = [query_dict[w] for w in seq.split()] # convert to word indices
onehot = np.zeros([len(w),len(query_dict)], np.float32)
for t in range(len(w)):
    onehot[t,w[t]] = 1

pred = z(x).eval({x:[onehot]})[0]
best = np.argmax(pred)
print(best)
print(seq, ":", intent_wl[best])

14
BOS flights from new york to seattle EOS : flight


_unchecked_

**任务 4: 使用所有隐藏状态进行序列分类**

在最后一个模型中, 我们看了最后一个 LSTM 块的输出。还有另一种模型, 在这里我们聚合所有 LSTM 块的输出, 并使用聚合输出到最终的 `Dense` 层。

因此, 您的任务将是替换 `C.layers.Fold` 带 `C.layers.Recurrence` 层函数。这是设置重复的明确方法。您将使用 `C.sequence.reduce_sum` 聚合来自 LSTM 块的所有中间输出。注意: 这与上次的模型不同, 我们只看最后一个 LSTM 块的输出。

所以请继续, 修改配置, 看看会发生什么。

如果一切顺利, 您将注意到改进的准确性 ( `metric` )。该解决方案将在之后提出, 但我们建议您不要查看解决方案。

In [26]:
# Replace the line with Fold operation
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Stabilizer(),
            C.layers.Fold(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_intents, name='classify')
        ])
# Enable these when done:
#do_train()
#do_test()

_unchecked_

聚合所有的中间状态可以提高相同次数的迭代的准确性, 而不会在计算时间上有任何显著的增加。

_unchecked_

**解决方案 4: 使用所有隐藏状态进行序列分类**

In [27]:
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Stabilizer(),
            C.sequence.reduce_sum(C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False)),
            C.layers.Dense(num_intents, name='classify')
        ])
    
do_train()
do_test()

Training 690477 parameters in 7 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 5]: [Training] loss = 0.193482 * 18004, metric = 4.07% * 18004 56.904s (316.4 samples/s);
Finished Epoch[2 of 5]: [Training] loss = 0.028983 * 17998, metric = 0.64% * 17998 73.820s (243.8 samples/s);
Finished Epoch[3 of 5]: [Training] loss = 0.059269 * 18000, metric = 1.24% * 18000 71.494s (251.8 samples/s);
Finished Epoch[4 of 5]: [Training] loss = 0.005993 * 18000, metric = 0.18% * 18000 61.856s (291.0 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 5]: [Training] loss = 0.000344 * 17998, metric = 0.01% * 17998 57.114s (315.1 samples/s);
Finished Evaluation [1]: Minibatch[1-23]: metric = 0.00% * 893;
